# Example notebook for supervised training of LTC cells for time-series prediction tasks
#### *(using the Pytorch implementation)*

### Import requirements

In [2]:
from ray.rllib.agents.ppo import PPOTrainer